In [1]:
word2vec_model = '/Users/iijima.s.ad/git/JASen/word2vec_100.txt'
comment_folder = '/Users/iijima.s.ad/git/article-extractor/articles/fox/drones/*'
comment_files = '/Users/iijima.s.ad/git/article-extractor/articles/fox/drones/*/comments.json'
output_folder = '/Users/iijima.s.ad/git/workspace/articles/fox/drones/'

train_files = '/Users/iijima.s.ad/git/article-extractor/articles/fox/*/*/NER_comments.txt'
source_file = '/Users/iijima.s.ad/git/workspace/text/NER_article2.txt'

In [21]:
import glob
from gensim.models import word2vec

word_list = []

for train_file in glob.glob(train_files):
    with open(train_file, 'r') as f:
        for sentence in f.readlines():
            word_list.append(sentence.strip().split(' '))

In [41]:
with open('articles/fox/drones/ner_train.txt', 'w') as f:
    f.writelines([' '.join(words) + '\n' for words in word_list])

In [8]:
# NER処理済みの記事をWord2vecで学習させる
from gensim.models import word2vec

word_list = []

with open(source_file, 'r') as f:
    for sentence in f.readlines():
        word_list.append([word for word in sentence.strip().split(' ') if word != '' and len(word) < 50])

model = word2vec.Word2Vec(word_list, workers=20, vector_size=100, min_count=5, window=5, epochs=5)
model.save("models/ner_article5.model")
model.wv.save_word2vec_format("models/wordvec/ner_article5.vec")

In [9]:
# NER処理済みのコメントを使ってWord2vecを追加学習させる
import gensim

word_add_list = []
with open('/Users/iijima.s.ad/git/workspace/articles/fox/ner_train2.txt', 'r') as f:
    for line in f.readlines():
        word_add_list.append([word for word in line.strip().split(' ') if word != '' and len(word) < 50])

model.train(word_add_list, total_examples=sum(len(s) for s in word_add_list), epochs=5)
model.save('models/ner_tuned.model')
model.wv.save_word2vec_format('models/wordvec/ner_tuned2.vec', binary=False)

In [68]:
import re

base_wv = gensim.models.KeyedVectors.load_word2vec_format('/Users/iijima.s.ad/git/JASen/word2vec_100.txt', binary=False)
word_trimed_list = []
for w in model.wv.index_to_key:
    if w in base_wv or re.match(r'\[\[\w+\]\]', w) is not None:
        word_trimed_list.append(f'{w} {" ".join([str(v) for v in model.wv[w]])}\n')

with open('ner_article_wv3.txt', 'w') as f:
    f.write(f'{len(word_trimed_list)} 100\n')
    f.writelines(word_trimed_list)

In [67]:
with open('/Users/iijima.s.ad/git/workspace/models/wordvec/ner_article_wv5.txt', 'r') as origin_wv:
    tmp_data = [line.strip().split(' ') for line in origin_wv.readlines()[1:]]

with open('/Users/iijima.s.ad/git/workspace/models/wordvec/ner_article_wv3.txt', 'r') as f:
    tmp_data2 = {line.strip().split(' ')[0] for line in f.readlines()[1:]}

diffrence_data = [l for l in tmp_data if l[0] not in tmp_data2]
with open('/Users/iijima.s.ad/git/workspace/models/sub/removed.txt', 'w') as f:
    f.write('\n'.join([' '.join(l) for l in diffrence_data]))


In [68]:
with open('/Users/iijima.s.ad/git/workspace/models/wordvec/ner_article_wv5.txt', 'r') as f:
    wv_data5 = f.readlines()[1:]

i, name = 0, 1
while i < len(diffrence_data):
    with open(f'/Users/iijima.s.ad/git/workspace/models/sub/ner_tmp{name}', 'w') as f:
        tmp_data4 = wv_data5 + [' '.join(l) + '\n' for l in diffrence_data[i:i+1000]]
        f.write(f'{len(tmp_data4)} 100\n')
        f.writelines(tmp_data4)
    i += 1000
    name += 1

In [71]:
import math

N = 10
vectors = diffrence_data[0:1000]
i, name = 0, 100
length = math.ceil(len(vectors) / N)

for j in range(N):
    with open(f'/Users/iijima.s.ad/git/workspace/models/sub/ner_tmp{name}', 'w') as f:
        tmp_data4 = wv_data5 + [' '.join(l) + '\n' for l in vectors[i:i+length]]
        f.write(f'{len(tmp_data4)} 100\n')
        f.writelines(tmp_data4)
        i += length
        name += 1


In [85]:
with open('/Users/iijima.s.ad/git/workspace/models/wordvec/ner_article_wv5.txt', 'r') as f:
    wv_data5 = f.readlines()[1:]

with open(f'/Users/iijima.s.ad/git/workspace/models/sub/ner_tmp0', 'w') as f:
    wv_data0 = [l for l in wv_data5 if len(l.strip().split(' ')) == 101]
    f.write(f'{len(wv_data0)} 100\n')
    f.writelines(wv_data0)

In [2]:
import gensim

model = gensim.models.Word2Vec.load('models/ner_article4.model')
model.wv.save_word2vec_format('models/wordvec/ner_article4.vec', binary=False)

In [26]:
tags = ['[[PERSON]]', '[[ORG]]', '[[NORP]]', '[[FAC]]', '[[LOC]]', '[[GPE]]', '[[PRODUCT]]', '[[WORK_OF_ART]]', '[[EVENT]]']

for word in ['ugly', 'miserable', 'negative', 'fearful', 'disagreeable', 'harsh', 'angry', 'resentful', 'improper', 'ill', 'upset', 'humble', 'deny', 'fear']:
    sorted_similarities = sorted([(model.wv.similarity(word, t), t) for t in tags], key=lambda t: t[0], reverse=True)
    print(word + ' ', end='')
    for s in sorted_similarities[:3]:
        print(f'{s[1]} {s[0]:.4f}, ', end='')
    print()
        


ugly [[WORK_OF_ART]] 0.0732, [[LOC]] 0.0466, [[PERSON]] 0.0353, 
miserable [[GPE]] 0.0505, [[EVENT]] 0.0347, [[NORP]] -0.0285, 
negative [[PRODUCT]] -0.0041, [[LOC]] -0.0104, [[GPE]] -0.0250, 
fearful [[NORP]] 0.0436, [[GPE]] 0.0141, [[LOC]] -0.0117, 
disagreeable [[PERSON]] 0.0244, [[LOC]] -0.0066, [[NORP]] -0.0354, 
harsh [[NORP]] 0.0467, [[LOC]] 0.0290, [[GPE]] 0.0290, 
angry [[NORP]] 0.1411, [[PERSON]] 0.0695, [[ORG]] 0.0462, 
resentful [[NORP]] 0.0689, [[LOC]] 0.0276, [[PERSON]] -0.0020, 
improper [[ORG]] -0.0121, [[PRODUCT]] -0.0140, [[NORP]] -0.0384, 
ill [[NORP]] 0.0357, [[EVENT]] -0.0085, [[PERSON]] -0.0271, 
upset [[PERSON]] 0.0926, [[NORP]] 0.0818, [[ORG]] 0.0670, 
humble [[WORK_OF_ART]] 0.1248, [[PERSON]] 0.1199, [[LOC]] 0.0655, 
deny [[ORG]] 0.0281, [[NORP]] 0.0189, [[PERSON]] 0.0047, 
fear [[LOC]] 0.0792, [[GPE]] 0.0569, [[NORP]] 0.0509, 
